# Lists

In [1]:
bob = ['Bob Smith', 42, 30000, 'software']
sue = ['Sue Jones', 45, 40000, 'hardware']

In [3]:
bob[0], sue[2]

('Bob Smith', 40000)

In [5]:
bob[0].split()[-1]

'Smith'

In [7]:
sue[2] *= 1.25
sue

['Sue Jones', 45, 50000.0, 'hardware']

# A database list

In [9]:
people = [bob, sue]
for person in people:
    print(person)

['Bob Smith', 42, 30000, 'software']
['Sue Jones', 45, 50000.0, 'hardware']


In [11]:
people[1][0]

'Sue Jones'

In [13]:
for person in people:
    print(person[0].split()[-1])
    person[2] *= 1.20

Smith
Jones


In [15]:
for person in people:
    print(person[2])

36000.0
60000.0


In [17]:
pays = [person[2] for person in people]
pays

[36000.0, 60000.0]

In [33]:
pays = map(lambda x: x[2], people)
pays = list(pays)
pays

[36000.0, 60000.0]

In [35]:
sum(pays)

96000.0

In [37]:
sum(person[2] for person in people)

96000.0

In [39]:
people.append(['Tom', 50, 0, None])

In [41]:
len(people)

3

In [43]:
for person in people:
    print(person)

['Bob Smith', 42, 36000.0, 'software']
['Sue Jones', 45, 60000.0, 'hardware']
['Tom', 50, 0, None]


# Field labels

In [48]:
NAME, AGE, PAY = range(3)
bob = ['Bob Smith', 42, 10000]

In [50]:
bob[NAME]

'Bob Smith'

In [52]:
PAY, bob[PAY]

(2, 10000)